# Assignment module 3

This assignment is about clustering. The dataset used contains information about the traffic flow during each 5 minute time interval in the year 2021.

### Package import 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import calinski_harabasz_score, silhouette_score, davies_bouldin_score
import sklearn.metrics.pairwise as dis_lib

### Data import

In [2]:
data_df = pd.read_csv("dataset_exercise_5_clustering_highway_traffic.csv",sep=";")

### Data preparation

In [3]:
# Sort the DataFrame 'data_df' by columns "Date" and "Interval_5"
data_df.sort_values(["Date", "Interval_5"])

#print(data_df)

# Extract unique dates from the sorted DataFrame
days = np.unique(data_df[['Date']].values.ravel())
#print(days)

# Calculate the total number of unique days
ndays = len(days)

# Group the DataFrame 'data_df' by the "Date" column
day_subsets_df = data_df.groupby(["Date"])

# Define the total number of 5-minute intervals in a day
nintvals = 288

# Create a matrix 'vectorized_day_dataset' filled with NaN values
vectorized_day_dataset = np.zeros((ndays, nintvals))
vectorized_day_dataset.fill(np.nan)

# Loop through each unique day
for i in range(0, ndays):
    # Get the DataFrame corresponding to the current day
    df_t = day_subsets_df.get_group(days[i])

    # Loop through each row in the current day's DataFrame
    for j in range(len(df_t)):
        # Get the current day's DataFrame
        df_t = day_subsets_df.get_group(days[i])

        # Extract the "Interval_5" and "flow" values and populate 'vectorized_day_dataset'
        vectorized_day_dataset[i, df_t.iloc[j]["Interval_5"]] = df_t.iloc[j]["flow"]

# Print the resulting 'vectorized_day_dataset'
print(vectorized_day_dataset)


# Create an array 'day_of_week' to store the day of the week for each unique date
day_of_week = np.zeros((ndays))

# Loop through each unique date
for i in range(0, ndays):
    # Parse the current date from a string to a datetime object
    day_dt = datetime.datetime.strptime(str(days[i]), '%Y%m%d')

    # Get the day of the week (1 for Monday, 2 for Tuesday, ..., 7 for Sunday)
    day_of_week[i] = day_dt.isoweekday()
    
nans_per_day = np.sum(np.isnan(vectorized_day_dataset),1)

[[ 39.  18.  26. ...  32.  39.  34.]
 [ 30.  32.  27. ...  44.  41.  39.]
 [ 36.  44.  52. ...  50.  45.  23.]
 ...
 [ 20.  34.  31. ...  38.  42.  36.]
 [ 36.  40.  25. ...  38.  56.  35.]
 [ 33.  32.  34. ... 130. 129. 117.]]


## Clustering

### Internal evaluation

In [4]:
vectorized_day_dataset_no_nans = vectorized_day_dataset[np.where(nans_per_day == 0)[0],:]
days_not_nans = days[np.where(nans_per_day == 0)[0]]

clusters = None
cluster_model_list = ["KMeans", "AgglomerativeClustering", "DBSCAN", "GaussianMixture"]
cluster_range_list = range(2, 11)

df_results_silhouette = pd.DataFrame(data=cluster_range_list, columns = ["Clusters"])
df_results_davies_bouldin = pd.DataFrame(data=cluster_range_list, columns = ["Clusters"])
df_results_calinski_harabasz = pd.DataFrame(data=cluster_range_list, columns = ["Clusters"])

for i in cluster_model_list:
    silhouette_list = []
    davies_bouldin_list = []
    calinski_harabasz_list = []    
    for j in cluster_range_list:
        n_clusters = j
        if i == "KMeans":
            clusters = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(vectorized_day_dataset_no_nans)
            cluster_labels = clusters.labels_
        elif i == "AgglomerativeClustering":
            clusters = AgglomerativeClustering(n_clusters=n_clusters,metric='euclidean', linkage='ward').fit(vectorized_day_dataset_no_nans)
            cluster_labels = clusters.labels_
        elif i == "DBSCAN":
            clusters = DBSCAN(eps=500, min_samples = 2).fit(vectorized_day_dataset_no_nans)
            cluster_labels = clusters.labels_
        elif i == "GaussianMixture":
            cluster_labels = GaussianMixture(n_components=n_clusters).fit(vectorized_day_dataset_no_nans).predict(vectorized_day_dataset_no_nans)
        else:
            print("Error: Incorrect cluster model")
        
        # Calculate the number of clusters by finding unique values in 'cluster_labels'
        n_clusters_t = len(np.unique(cluster_labels))
        
        # Calculate the Silhouette Score
        SC_score = silhouette_score(vectorized_day_dataset_no_nans, cluster_labels)
        silhouette_list.append(SC_score)
        # Silhouette Score measures the quality of clusters, higher values indicate better separation.

        # Calculate the Davies-Bouldin Score
        DB_score = davies_bouldin_score(vectorized_day_dataset_no_nans, cluster_labels)
        davies_bouldin_list.append(DB_score)
        # Davies-Bouldin Score measures the average similarity between each cluster and its most similar cluster, lower values indicate better separation.

        # Calculate the Calinski-Harabasz Score
        CH_score = calinski_harabasz_score(vectorized_day_dataset_no_nans, cluster_labels)
        calinski_harabasz_list.append(CH_score)
        # Calinski-Harabasz Score measures the ratio of between-cluster variance to within-cluster variance, higher values indicate better separation.
        
    df_results_silhouette[i] = silhouette_list
    df_results_davies_bouldin[i] = davies_bouldin_list
    df_results_calinski_harabasz[i] = calinski_harabasz_list 
               
print(df_results_silhouette) 
print(df_results_davies_bouldin)
print(df_results_calinski_harabasz)


C:\Users\tomko\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\tomko\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\tomko\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\tomko\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Window

   Clusters    KMeans  AgglomerativeClustering    DBSCAN  GaussianMixture
0         2  0.307621                 0.290945 -0.027721         0.307557
1         3  0.300536                 0.268112 -0.027721         0.279376
2         4  0.279691                 0.257637 -0.027721         0.228264
3         5  0.223437                 0.263778 -0.027721         0.232653
4         6  0.236851                 0.250363 -0.027721         0.246955
5         7  0.237882                 0.243157 -0.027721         0.220194
6         8  0.232969                 0.239635 -0.027721         0.220029
7         9  0.223869                 0.233039 -0.027721         0.182827
8        10  0.207933                 0.215062 -0.027721         0.176048
   Clusters    KMeans  AgglomerativeClustering    DBSCAN  GaussianMixture
0         2  1.267725                 1.279138  2.379572         1.269963
1         3  1.192975                 1.313811  2.379572         1.319473
2         4  1.157241                 

### Parameter optimalisation

In [5]:
n_clusters = 2

#clusters = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto", algorithm="elkan").fit(vectorized_day_dataset_no_nans) # Parameter tested: {“lloyd”, “elkan”, “auto”, “full”}
#clusters = AgglomerativeClustering(n_clusters=n_clusters,metric='euclidean', linkage='ward').fit(vectorized_day_dataset_no_nans) # check the parameters at https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
#clusters = DBSCAN(eps=500, min_samples = 2).fit(vectorized_day_dataset_no_nans) # check the parameters at https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

#if clusters is not None:
#  cluster_labels = clusters.labels_

cluster_labels = GaussianMixture(n_components=n_clusters, covariance_type="spherical").fit(vectorized_day_dataset_no_nans).predict(vectorized_day_dataset_no_nans) #covariance_type{‘full’, ‘tied’, ‘diag’, ‘spherical’}


# Calculate the Silhouette Score
SC_score_improve = silhouette_score(vectorized_day_dataset_no_nans, cluster_labels)
# Silhouette Score measures the quality of clusters, higher values indicate better separation.

# Calculate the Davies-Bouldin Score
DB_score_improve = davies_bouldin_score(vectorized_day_dataset_no_nans, cluster_labels)
# Davies-Bouldin Score measures the average similarity between each cluster and its most similar cluster, lower values indicate better separation.

# Calculate the Calinski-Harabasz Score
CH_score_improve = calinski_harabasz_score(vectorized_day_dataset_no_nans, cluster_labels)
# Calinski-Harabasz Score measures the ratio of between-cluster variance to within-cluster variance, higher values indicate better separation.

# Print the computed cluster quality scores
print('Silhouette Score:', SC_score_improve)
print('Davies-Bouldin Score:', DB_score_improve)
print('Calinski-Harabasz Score:', CH_score_improve)

Silhouette Score: 0.07376899788914677
Davies-Bouldin Score: 2.6764334562222967
Calinski-Harabasz Score: 30.0559655749271


C:\Users\tomko\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


### External evaluation with short-term prediction: Import evaluation data

In [6]:
# Read the evaluation dataset from a CSV file
data_eval_df = pd.read_csv("evaluation_dataset_exercise_5_clustering_highway_traffic.csv", sep=";")

# Sort the evaluation DataFrame by columns "Date" and "Interval_5"
data_eval_df.sort_values(["Date", "Interval_5"])

# Extract unique dates from the sorted evaluation DataFrame
days_eval = np.unique(data_eval_df[['Date']].values.ravel())
# Calculate the total number of unique days in the evaluation dataset
ndays_eval = len(days_eval)

# Group the evaluation DataFrame by the "Date" column
day_eval_subsets_df = data_eval_df.groupby(["Date"])

# Initialize a matrix 'vectorized_day_dataset_eval' filled with NaN values
vectorized_day_dataset_eval = np.zeros((ndays_eval, nintvals))
vectorized_day_dataset_eval.fill(np.nan)
# This section initializes a 2D array to store the evaluation dataset and fills it with NaN values.

# Loop through each unique day in the evaluation dataset
for i in range(0, ndays_eval):
    # Get the DataFrame corresponding to the current day
    df_t = day_eval_subsets_df.get_group(days_eval[i])

    # Loop through each row in the current day's DataFrame
    for j in range(len(df_t)):
        # Get the current day's DataFrame (this line is redundant)
        df_t = day_eval_subsets_df.get_group(days_eval[i])

        # Extract the "Interval_5" and "flow" values and populate 'vectorized_day_dataset_eval'
        vectorized_day_dataset_eval[i, df_t.iloc[j]["Interval_5"]] = df_t.iloc[j]["flow"]

# Print the resulting 'vectorized_day_dataset_eval'
print(vectorized_day_dataset_eval)


# Calculate the total number of NaN values in the evaluation dataset
print('Number of NaNs:', np.sum(np.isnan(vectorized_day_dataset_eval)))

# Calculate the rate of NaN values in the evaluation dataset
print('Rate of NaNs:', np.sum(np.isnan(vectorized_day_dataset_eval)) / (ndays_eval * nintvals))

# Calculate the number of days with missing values
nans_per_day_eval = np.sum(np.isnan(vectorized_day_dataset_eval), 1)
print('Number of days with missing values:', np.size(np.where(nans_per_day_eval > 0)))

# Filter out days with no missing values and create a new dataset
vectorized_day_dataset_no_nans_eval = vectorized_day_dataset_eval[np.where(nans_per_day_eval == 0)[0], :]
days_not_nans_eval = days_eval[np.where(nans_per_day_eval == 0)[0]]

# Calculate the final number of days in the evaluation dataset after removing missing values
print('Final number of days in evaluation dataset:', len(days_not_nans_eval))

# Print the list of days in the evaluation dataset with no missing values
print('List of days without missing values:', days_not_nans_eval)

# Calculate the total number of days in the filtered evaluation dataset
ndays_eval_not_nans = len(days_not_nans_eval)

[[16.74 17.57 16.94 ... 19.43 19.88 19.73]
 [19.17 19.26 20.98 ... 19.34 21.22 19.79]
 [19.69 19.39 19.76 ... 20.51 20.5  19.6 ]
 ...
 [19.65 22.59 20.8  ... 19.23 20.32 20.18]
 [19.75 19.96 20.95 ... 17.95 19.14 19.55]
 [20.82 20.28 20.5  ... 19.72 20.64 20.23]]
Number of NaNs: 96
Rate of NaNs: 0.004166666666666667
Number of days with missing values: 11
Final number of days in evaluation dataset: 69
List of days without missing values: [20220108 20220109 20220131 20220204 20220209 20220210 20220211 20220223
 20220226 20220227 20220302 20220304 20220305 20220306 20220310 20220314
 20220315 20220321 20220323 20220326 20220403 20220406 20220416 20220418
 20220421 20220422 20220425 20220427 20220428 20220503 20220505 20220514
 20220519 20220521 20220522 20220526 20220530 20220601 20220603 20220609
 20220616 20220619 20220623 20220628 20220704 20220711 20220712 20220904
 20220910 20220911 20220920 20220921 20220925 20220927 20220929 20220930
 20221005 20221022 20221024 20221114 20221116 20

### Model evaluation on evaluation data

In [7]:
centroids = []
clusters = None
n_clusters_t = len(np.unique(cluster_labels))
n_clusters = 5

#Clustering functions for model evaluation:
#clusters = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(vectorized_day_dataset_no_nans) # Parameter tested: {“lloyd”, “elkan”, “auto”, “full”}
clusters = AgglomerativeClustering(n_clusters=n_clusters,metric='euclidean', linkage='ward').fit(vectorized_day_dataset_no_nans) # check the parameters at https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
#clusters = DBSCAN(eps=500, min_samples = 2).fit(vectorized_day_dataset_no_nans) # check the parameters at https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

if clusters is not None:
    cluster_labels = clusters.labels_
    print(clusters)

#cluster_labels = GaussianMixture(n_components=n_clusters, covariance_type="tied").fit(vectorized_day_dataset_no_nans).predict(vectorized_day_dataset_no_nans) #covariance_type{‘full’, ‘tied’, ‘diag’, ‘spherical’}


# Define a function to find the closest centroid to a new data point within a specified day-time interval range
def find_the_closest_centroid(centroids, new_day, from_interval: int, to_interval: int):
    closest_centroid = None
    closest_dist = None

    # Iterate through each centroid
    for i in range(0, len(centroids)):
        # Calculate the Euclidean distance between the centroid and the new data point
        ed_t = dis_lib.paired_distances(centroids[i], new_day, metric='euclidean')

        # Check if the current centroid is closer than the previously closest one
        if closest_centroid is None or closest_dist > ed_t:
            closest_centroid = i
            closest_dist = ed_t

    return closest_centroid

# Initialize a list to store centroid data
centroids = []

# Calculate centroids for each cluster
for i in range(0, n_clusters_t):
    centroid = np.nanmean(vectorized_day_dataset_no_nans[np.where(cluster_labels == i)[0], :], 0).reshape(1, nintvals)
    centroids.append(centroid)

# Define the number of past intervals to consider for classification
n_past_intervals_for_classification = 5

# Initialize variables to calculate accuracy metrics
total_mae = 0
total_mape = 0
prediction_counts = 0


#print("Number of NANs is: ", np.sum(np.isnan(vectorized_day_dataset_no_nans_eval)))


# Loop through each day in the evaluation dataset with no missing values
for i in range(0, ndays_eval_not_nans):
    # Loop through intervals from n_past_intervals_for_classification to nintvals - 1
    for j in range(n_past_intervals_for_classification, nintvals - 1):
        # Find the closest centroid for the current data point
        centroid_index = find_the_closest_centroid(centroids, vectorized_day_dataset_no_nans_eval[i].reshape(1, nintvals), j - n_past_intervals_for_classification, j)

        # Predict the value for the next interval
        predicted_value = centroids[centroid_index][0, j + 1]

        # Calculate Mean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE)
        mae_t = abs(predicted_value - vectorized_day_dataset_no_nans_eval[i][j + 1])
        mape_t = abs(predicted_value - vectorized_day_dataset_no_nans_eval[i][j + 1]) / float(vectorized_day_dataset_no_nans_eval[i][j + 1])

        # Accumulate MAE, MAPE, and count of predictions
        total_mae += mae_t
        total_mape += mape_t
        prediction_counts += 1

# Calculate and print the prediction accuracy metrics
print('Prediction accuracy MAE:', total_mae / prediction_counts)
print('Prediction accuracy MAPE:', total_mape / prediction_counts)


AgglomerativeClustering(metric='euclidean', n_clusters=5)
Prediction accuracy MAE: 153.37647598101387
Prediction accuracy MAPE: 9.84753902348205
